# Dataset

In [ ]:
import json
import pandas as pd
import pymysql
from sqlalchemy import create_engine
from sklearn.cross_validation import train_test_split
from sklearn import metrics
import pickle
from sklearn import ensemble
from sklearn import datasets
from sklearn.utils import shuffle
from sklearn.metrics import mean_squared_error
pd.set_option('display.max_columns', 500)

In [ ]:
with open("credentials.json") as f:
    credentials = json.loads(f.read())
    
    host = credentials["host"]
    user = credentials["db_user"]
    password = credentials["db_pass"]
    db = credentials["db_name"]

In [ ]:
# Automate this query for every route for every direction

engine = create_engine(f"mysql+pymysql://{user}:{password}@{host}:3306/{db}")

df = pd.read_sql_query('SELECT * FROM trips_2017 WHERE lineid = "46A" AND direction = 2', engine)
df.head()

In [ ]:
# Replace missing actual time departure values with planned values

df.actualtime_dep.fillna(df.plannedtime_dep, inplace=True)
df.head()

In [ ]:
# Remove rows with missing values for actual time arrival as we cannot safely assume these are as planned

df = df[pd.notnull(df['actualtime_arr'])]
df.head()

In [ ]:
# Create a new column for trip duration

df['trip_duration'] = df['actualtime_arr'] - df['actualtime_dep']
df.head()

In [ ]:
# Create a new column with the hour of the day the trip took place

df['actualtime_dep_H'] = round(df['actualtime_dep']/3600)
df.head()

In [ ]:
# Hour of day of actual time arrival

df['actualtime_arr_H'] = round(df['actualtime_arr']/3600)
df.head()

In [ ]:
# Average hour of the day of the journey

df['avg_H'] = (df['actualtime_dep_H'] + df['actualtime_arr_H']) / 2
df.head()

In [ ]:
# Convert this to an integer

df['avg_H'] = df['avg_H'].astype(int)
df.head()

In [ ]:
# Creating column solely for the dates to correlate with the dates column on the historical weather data table

df['time'] = df['timestamp'] + df['avg_H'] * 3600
df.time

In [ ]:
# Removing suppressed rows where suppressed = 1.0

df = df.query('suppressed != 1.0')

In [ ]:
# How many rows are we working with

df.index = range(len(df))

In [ ]:
# Creating columns from timestamp for further processing

df['dayofweek'] = df['timestamp']
df['monthofyear'] = df['timestamp']

In [ ]:
# Converting the unix time to datetime format

df.dayofweek = pd.to_datetime(df['dayofweek'], unit='s')
df.monthofyear = pd.to_datetime(df['monthofyear'], unit='s')

In [ ]:
# Converting datetime to name of weekday, and to name of month (in separate columns)

df['dayofweek'] = df['dayofweek'].dt.weekday_name
df['monthofyear'] = df['monthofyear'].dt.month

In [ ]:
# Creating dummy variables for weekday names and name of month

df_dayofweek_dummies = pd.get_dummies(df['dayofweek'])

In [ ]:
# Removing rows not in the month of March
# We chose March as we felt it was the best representation of a typical 'school' month

df = df.query('monthofyear == 3')

In [ ]:
# Add day of week columns for each day 

df1 = pd.concat([df, df_dayofweek_dummies], axis=1, join_axes=[df.index])

In [ ]:
df1.head()

In [ ]:
# Pull weather data from database

df2 = pd.read_sql_query('SELECT * FROM DarkSky_historical_weather_data WHERE year = 2017 AND month = 3', engine)
df2.head()

In [ ]:
# Replace values for clarity purposes i.e. we care if it is cloudy; cloudy-day and cloudy-night distinctions are irrelevant

d = {'clear-day':'clear','clear-night':'clear','partly-cloudy-day':'partly-cloudy','partly-cloudy-night':'partly-cloudy'}
df2 = df2.replace(d)

In [ ]:
df2.rename(columns={'day_of_week': 'dayofweek', 'month': 'monthofyear'}, inplace=True)

In [ ]:
# Mergin bus and weather data on timestamp

df3 = pd.merge(df1, df2, on=['time'])

In [ ]:
df3.head()

In [ ]:
# Selecting 'useful' features for analysis

df3 = df3[['avg_H', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday', 'temp', 'precip_intensity','trip_duration']]

In [ ]:
# Trip duration is in seconds, convert to minutes and round to the nearest integer

df3['trip_duration'] = round(df3['trip_duration']/60)

In [ ]:
df3['trip_duration'] = df3['trip_duration'].astype(int)

In [ ]:
# Easier to work with whole number for temperature

df3['temp'] = round(df3['temp'])

In [ ]:
df3['temp'] = df3['temp'].astype(int)

In [ ]:
# Our dataframe is ready for processing

df3.head()

In [ ]:
df3.shape

# Preprocessing
You can see that our dataset has eleven columns. The task is to predict the trip duration (last column) based on the day of the week, the time of the day and the weather conditions (temperature and rain intesity). The next step is to split our dataset into attributes and labels. 

In [ ]:
# Assign data from first ten columns to X variable
# Descriptive features

X = df3.iloc[:, 0:10]

# Assign data from last column to y variable
# Target feature

y = df3['trip_duration']

# Training the GBR model

# Parameters
### n_estimators : int (default=100)
    The number of boosting stages to perform. 
    Gradient boosting is fairly robust to over-fitting so a large number usually results in better performance.

### max_depth : integer, optional (default=3)
    maximum depth of the individual regression estimators. 
    The maximum depth limits the number of nodes in the tree. 
    Tune this parameter for best performance; the best value depends on the interaction of the input variables.
    
### min_samples_split : int, float, optional (default=2)
    The minimum number of samples required to split an internal node:
    If int, then consider min_samples_split as the minimum number.
    If float, then min_samples_split is a percentage and ceil(min_samples_split * n_samples) are the minimum number of samples for each split.
        Changed in version 0.18: Added float values for percentages.

### learning_rate : float, optional (default=0.1)
    learning rate shrinks the contribution of each tree by learning_rate. 
    There is a trade-off between learning_rate and n_estimators.

### loss : {‘deviance’, ‘exponential’}, optional (default=’deviance’)
    loss function to be optimized. 
    ‘deviance’ refers to deviance (= logistic regression) for classification with probabilistic outputs. 
    For loss ‘exponential’ gradient boosting recovers the AdaBoost algorithm.

In [ ]:
# Fit regression model
# Peter, maybe look at automating trying out a few different parameters and choosing the best one?

params = {'n_estimators': 600, 'max_depth': 4, 'min_samples_split': 2,
          'learning_rate': 0.02, 'loss': 'ls'}
gbr = ensemble.GradientBoostingRegressor(**params)

gbr.fit(X, y)

In [ ]:
# Compute the importance of each feature based on the model

pd.DataFrame({'feature': X.columns, 'importance': gbr.feature_importances_})

In [ ]:
# Generate predictions for the dataset

pred = gbr.predict(X)

In [ ]:
predictions = pd.DataFrame(pred)
predictions.rename(columns={0:'estimated_time'}, inplace=True )
predictions['estimated_time'] = round(predictions['estimated_time'])
predictions['estimated_time'] = predictions['estimated_time'].astype(int)
predictions.head()

In [ ]:
# Check the error rate
# Peter, if this is more than 9, maybe get the script to flag this pickle, possible with the file name?

print(metrics.mean_absolute_error(y,predictions)) 

In [ ]:
# Storing the model trained on the full data set to a pickle file
# This will need to be automated to contain route and direction

pkl_filename = "GBR_March_2017_46A_2.pkl"
with open(pkl_filename, 'wb') as file:  
    pickle.dump(gbr, file)